In [448]:
import tweepy
import pandas as pd
client = tweepy.Client("AAAAAAAAAAAAAAAAAAAAAGvbawEAAAAAxvxnJu9E%2F%2FPCe5Oz0irJC5BDvZM%3DEsqky8zPsYhadIeHKzCVsaB0UXGz28FFb2UBqtnNDOJkmfAkfN")
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
import re
import flair
import yfinance as yf

In [212]:
elon_musk=44196397

In [216]:
def user_tweets(user_id, start_time, end_time, iterations):
    datelist = pd.date_range(start_time, end_time, periods=iterations).tolist()
    tweets=[]
    date=[]
    a=0
    for time in datelist:
        if a>1:
            user = client.get_users_tweets(user_id, max_results=100, start_time=start, end_time=time, tweet_fields=["created_at"])
            if type(user.data) != type(None):
                for i in user.data: 
                    tweets.append(str(i))
                    date.append(i.created_at)
        start=time
        a=a+1    
    tweet_df=pd.DataFrame({'time':date, 'tweet':tweets})
    tweet_df=tweet_df.sort_values(by='time', ascending=True).set_index('time')
    return tweet_df
   
    

In [217]:
user_tweets(elon_musk,'2021-01-01', '2022-01-01', 3)

,tweet
time,


In [576]:
elon = client.get_users_tweets(elon_musk, max_results=100, tweet_fields=["created_at"])
t=[]
d=[]
for i in elon.data:
    t.append(str(i))
    d.append(i.created_at)
elon_df=pd.DataFrame({'time':d, 'tweet':t})
elon_df['time']=elon_df['time'].round('1min')
elon_df=elon_df.sort_values(by='time', ascending=True).set_index('time')
elon_df

,tweet
time,
2022-04-03 06:19:00+00:00,Peace. Peace? I hate the word. Those who do c...
2022-04-03 06:23:00+00:00,@WholeMarsBlog There will come a time in the f...
2022-04-03 06:54:00+00:00,I am inspired by curiosity. \n\nThat is what d...
2022-04-03 07:15:00+00:00,@JuicyCreativ3 Old people don’t change their m...
2022-04-03 07:16:00+00:00,@JuicyCreativ3 George Lucas once told me that ...
2022-04-03 09:15:00+00:00,https://t.co/VlIZsJyH4F
2022-04-03 09:46:00+00:00,@PopMech Good summary
2022-04-03 18:51:00+00:00,@GerberKawasaki We aim for 30% GM or ~10% prof...
2022-04-03 19:03:00+00:00,@Erdayastronaut @Tesla Raptor pump power is ov...


In [28]:
response = client.get_users_tweets(elon_musk, max_results=100, tweet_fields=["created_at"])
for tweet in response.data:
    print(tweet)

Berlin rocks
@Erdayastronaut @Tesla Raptor pump power is over 100MW per engine &amp; 33 engines, means over 3GW. Not even remotely possible for electric motors &amp; batteries to compete.
@GerberKawasaki We aim for 30% GM or ~10% profitability, all costs included
@PopMech Good summary
https://t.co/VlIZsJyH4F
@JuicyCreativ3 George Lucas once told me that the only reason Star Wars got made is because the old studio heads died
@JuicyCreativ3 Old people don’t change their minds, with rare exception, they just die. Without death, there would not be change.
I am inspired by curiosity. 

That is what drives me. 

So let us expand the scope &amp; scale of consciousness so that we may aspire to understand the Universe.
@WholeMarsBlog There will come a time in the future where people will be surprised that we let practically anyone drive a 2 ton death machine anywhere they want
Peace. Peace?  I hate the word. Those who do care about peace (myself aspirationally included) don’t need to hear it. A

In [569]:
def basic_count(query):
    count_df=pd.DataFrame(columns=['date', 'count'])
    response = client.get_recent_tweets_count(query, granularity="hour")
    for count in response.data:
        data={'date':count['end'], 'count':count['tweet_count']}
        count_df=count_df.append(data, ignore_index=True)
    plt=hvplot.plot(count_df, x='date', y='count', kind='line')
    return plt

In [562]:
query='tesla OR tsla'

In [91]:
pos_query='#bitcoin (Moon OR rocket OR bullish OR up OR positive OR gain) -bearish -lose -crash -down -negative'

In [92]:
basic_count(pos_query)

:Curve   [date]   (count)

In [563]:
basic_count(query)

:Curve   [date]   (count)

In [60]:
datelist = pd.date_range(start, end=datetime.today(), periods=100).tolist()

In [286]:
sentiment_model = flair.models.TextClassifier.load('en-sentiment')
def tweets(start_time, end_time, iterations, query):
    datelist = pd.date_range(start_time, end_time, periods=iterations).tolist()
    tweets=[]
    date=[]
    prob=[]
    sentim=[]
    a=0
    for time in datelist:
        if a>1:
            #Fetching Tweets
            response = client.search_recent_tweets(query, max_results=100, start_time=start, end_time=time, tweet_fields=["created_at"])
            response.data
            if type(response.data) != type(None):
                #Data Cleaning
                for i in response.data:
                    tw=i
                    tw=str(i)
                    tw=tw.lower()
                    tw=re.sub(r"http\S+", '',tw)
                    tw=re.sub(r"www.\S+",'',tw)
                    tw=re.sub("[^A-Za-z0-9]"," ",tw)
                    tw=tw.split()
                    tw=' '.join(word for word in tw)
                    #Sentiment analysis
                    if len(tw) > 0:
                        sentence = flair.data.Sentence(tw)
                        sentiment_model.predict(sentence)
                        probability=sentence.labels[0].score
                        sentiment=sentence.labels[0].value
                        tweets.append(tw)
                        date.append(i.created_at)
                        prob.append(probability)
                        sentim.append(sentiment)
        start=time
        a=a+1
        
    tweet_df=pd.DataFrame({'time':date, 'tweet':tweets, 'probability':prob, 'sentiment':sentim})
    tweet_df['time']=tweet_df['time'].round('1min')
    tweet_df=tweet_df.sort_values(by='time', ascending=True).set_index('time')
    return tweet_df

2022-04-05 16:36:02,992 loading file C:\Users\peeta\.flair\models\sentiment-en-mix-distillbert_4.pt


In [270]:
twt='i hate you'
sentiment_model = flair.models.TextClassifier.load('en-sentiment')
sentence = flair.data.Sentence(twt)
sentiment_model.predict(sentence)
sentence.labels

2022-04-05 16:26:26,334 loading file C:\Users\peeta\.flair\models\sentiment-en-mix-distillbert_4.pt


[NEGATIVE (0.9997)]

In [189]:
tw='@@@@ %%%%##@)(*%!)%!)@%&!_%!_%&!@%!_%@ HeLLo_wOrLd https://t.co/eoTT9FXVr4 _()&@!_$!@+!@$)@!@'
tw=tw.lower()
tw=re.sub(r"http\S+", '',tw)
tw=re.sub(r"www.\S+",'',tw)
tw=re.sub("[^A-Za-z0-9]"," ",tw)
tw=tw.split()
tw=' '.join(word for word in tw)
print(tw)

hello world


In [287]:
tweet=tweets('2022-03-30', '2022-04-05', 168, 'tesla OR tsla -is:retweet lang:en')

In [288]:
tweet

,tweet,probability,sentiment
time,,,
2022-03-30 01:42:00+00:00,doge yog elonmusk spacex nasa tesla billym2k m...,0.993958,POSITIVE
2022-03-30 01:42:00+00:00,rt elonmusk sawyermerritt the uaw stole millio...,0.734933,NEGATIVE
2022-03-30 01:42:00+00:00,rt femalemsktr did ya ll read the article abou...,0.999954,NEGATIVE
2022-03-30 01:42:00+00:00,elonmusk bbcsciencenews fossil fuel is still d...,0.992671,POSITIVE
2022-03-30 01:42:00+00:00,yesterday i had only 10 btc today 100 000 yeah...,0.956366,NEGATIVE
...,...,...,...
2022-04-05 00:00:00+00:00,rt cosicomarmee shelbychurch tesla on behalf o...,0.770287,POSITIVE
2022-04-05 00:00:00+00:00,rt fatewalkr who is the motherfucker in maryla...,0.758968,POSITIVE
2022-04-05 00:00:00+00:00,rt papitrumpo now let s get started on those t...,0.957656,POSITIVE


In [289]:
tweet.to_csv('tesla.csv')

In [588]:
test=tweet[tweet['probability']>0.95]

In [589]:
test

,tweet,probability,sentiment
time,,,
2022-03-30 01:42:00+00:00,doge yog elonmusk spacex nasa tesla billym2k m...,0.993958,POSITIVE
2022-03-30 01:42:00+00:00,rt femalemsktr did ya ll read the article abou...,0.999954,NEGATIVE
2022-03-30 01:42:00+00:00,elonmusk bbcsciencenews fossil fuel is still d...,0.992671,POSITIVE
2022-03-30 01:42:00+00:00,yesterday i had only 10 btc today 100 000 yeah...,0.956366,NEGATIVE
2022-03-30 01:42:00+00:00,very ethereum holder will receive a full bag o...,0.997815,NEGATIVE
...,...,...,...
2022-04-05 00:00:00+00:00,teslanakamoto with tesla stock in your account...,0.976579,NEGATIVE
2022-04-05 00:00:00+00:00,rt papitrumpo now let s get started on those t...,0.957656,POSITIVE
2022-04-05 00:00:00+00:00,rt papitrumpo now let s get started on those t...,0.957656,POSITIVE


In [590]:
test=tweet.pivot_table('probability',['time','tweet'], 'sentiment')

In [591]:
test

sentiment                                                                     NEGATIVE  \
time                      tweet                                                          
2022-03-30 01:42:00+00:00 anyone else think it s weird how obsessed some ...  0.902402   
                          bitcoin is getting ready to take off to the moo...       NaN   
                          cgdavemac tesla raj klwtts sjvtesla elonmusk it...       NaN   
                          christo38106265 gokulram84 doge yog elonmusk sp...       NaN   
                          dennis p tesla right good good you can roadtrip...  0.981042   
...                                                                                ...   
2022-04-05 00:00:00+00:00 rt yamikawadays                                          NaN   
                          tesla is a trex gerberkawasaki watch our rod wi...       NaN   
                          teslanakamoto with tesla stock in your account ...  0.976579   
                          the same reason for one s rise to stardom on wa...  0.953623   
                          thegoodliars damn that russian propaganda machi...       NaN   

sentiment                                                                     POSITIVE  
time                      tweet                                                         
2022-03-30 01:42:00+00:00 anyone else think it s weird how obsessed some ...       NaN  
                          bitcoin is getting ready to take off to the moo...  0.596167  
                          cgdavemac tesla raj klwtts sjvtesla elonmusk it...  0.996721  
                          christo38106265 gokulram84 doge yog elonmusk sp...  0.960432  
                          dennis p tesla right good good you can roadtrip...       NaN  
...                                                                                ...  
2022-04-05 00:00:00+00:00 rt yamikawadays                                     0.964495  
                          tesla is a trex gerberkawasaki watch our rod wi...  0.953146  
                          teslanakamoto with tesla stock in your account ...       NaN  
                          the same reason for one s rise to stardom on wa...       NaN  
                          thegoodliars damn that russian propaganda machi...  0.783115  

[13480 rows x 2 columns]

In [592]:
test=test.groupby('time').count()


In [593]:
test

sentiment,NEGATIVE,POSITIVE
time,,
2022-03-30 01:42:00+00:00,18,12
2022-03-30 01:43:00+00:00,21,20
2022-03-30 02:32:00+00:00,7,4
2022-03-30 02:33:00+00:00,15,11
2022-03-30 02:34:00+00:00,18,13
...,...,...
2022-04-04 23:07:00+00:00,12,20
2022-04-04 23:08:00+00:00,12,18
2022-04-04 23:58:00+00:00,9,2


In [594]:
test=test.reset_index()
test['time']=test['time'].round('60min')
test=test.groupby('time').sum()

In [595]:
test

sentiment,NEGATIVE,POSITIVE
time,,
2022-03-30 02:00:00+00:00,39,32
2022-03-30 03:00:00+00:00,89,70
2022-03-30 04:00:00+00:00,37,42
2022-03-30 05:00:00+00:00,42,43
2022-03-30 06:00:00+00:00,41,43
...,...,...
2022-04-04 20:00:00+00:00,48,49
2022-04-04 21:00:00+00:00,98,93
2022-04-04 22:00:00+00:00,43,47


In [596]:
#test=test.groupby(test.index.to_period('H')).sum()
#test.groupby(pd.Grouper(key='time',freq='H')).sum()
test.hvplot(x='time',y=['POSITIVE','NEGATIVE'])

:NdOverlay   [sentiment]
   :Curve   [time]   (value)

In [623]:
test['pct']=(test['POSITIVE']/(test['POSITIVE']+test['NEGATIVE']))*2000

In [648]:
test

sentiment,NEGATIVE,POSITIVE,pct
time,,,
2022-03-30 02:00:00+00:00,39,32,901.408451
2022-03-30 03:00:00+00:00,89,70,880.503145
2022-03-30 04:00:00+00:00,37,42,1063.291139
2022-03-30 05:00:00+00:00,42,43,1011.764706
2022-03-30 06:00:00+00:00,41,43,1023.809524
...,...,...,...
2022-04-04 20:00:00+00:00,48,49,1010.309278
2022-04-04 21:00:00+00:00,98,93,973.821990
2022-04-04 22:00:00+00:00,43,47,1044.444444


In [624]:
test.hvplot(x='time', y='pct')

:Curve   [time]   (pct)

In [654]:
tsla_stock.hvplot(y='Close')

:Curve   [Datetime]   (Close)

In [643]:
tsla = yf.Ticker("TSLA")
tsla_stock = tsla.history(
    start='2022-04-05',
    end='2022-04-06',
    interval='1m'
)

In [644]:
tsla_stock=tsla_stock[tsla_stock['Volume']!=0]

In [645]:
tsla_stock

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2022-04-04 10:01:00-04:00,1095.199951,1096.900024,1093.264038,1096.574951,159220,0,0
2022-04-04 10:02:00-04:00,1096.334595,1098.250000,1095.500000,1098.250000,164599,0,0
2022-04-04 10:03:00-04:00,1097.795044,1098.725586,1096.430054,1097.050049,99794,0,0
2022-04-04 10:04:00-04:00,1097.530029,1101.000000,1097.020020,1098.300049,214271,0,0
2022-04-04 10:05:00-04:00,1098.788208,1100.989990,1096.310059,1099.979980,132287,0,0
...,...,...,...,...,...,...,...
2022-04-04 15:55:00-04:00,1142.000000,1142.679932,1140.170044,1142.310059,97514,0,0
2022-04-04 15:56:00-04:00,1142.300049,1142.310059,1141.010010,1141.300049,78255,0,0
2022-04-04 15:57:00-04:00,1141.510010,1143.094971,1141.280029,1143.094971,93876,0,0
